### Basic Quantum Entanglement for Ekert Protocol using Quantum Gates
#### This implementation generates entanglement between 2 qubits using Quantum Gates for the Ekert Protocol. Next, the idea is to create a Quantum Network with that QKD Protocol.
_ref: https://qiskit.org/textbook/ch-gates/multiple-qubits-entangled-states.html (adapted by D-Cryp7 for Netsquid 1.1.6)_

In [3]:
import netsquid as ns
from random import randint
from hashlib import sha256
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

def random_basis():
    r = randint(0, 1)
    return ns.Z if r else ns.X

def measure(q, obs):
    measurement_result, prob = ns.qubits.measure(q, obs)
    if measurement_result == 0:
        state = "|0>" if obs == ns.Z else "|+>"
    else:
        state = "|1>" if obs == ns.Z else "|->"
    # print(f"Measured {state} with probability {prob:.1f}")
    return ("Z" if obs == ns.Z else "X"), measurement_result, prob

ns.sim_reset()
alice_measures, bob_measures = [], []
for i in range(256):
    qubits = ns.qubits.create_qubits(2)
    q1, q2 = qubits[0], qubits[1]
    ns.qubits.operate(q1, ns.H) # Hadamard gate in q1
    ns.qubits.combine_qubits([q1, q2]) # Combine states. ej: |0> |0> -> |00>
    ns.qubits.operate([q1, q2], ns.CX) # CNOT gate in q1, q2 (Entanglement)
    # print(ns.qubits.reduced_dm([q1, q2]))
    alice_measures.append(measure(q1, random_basis())) # Alice measure, colapse the Entanglement
    bob_measures.append(measure(q2, random_basis())) # Bob measure with 100% in the same basis

alice_match, bob_match = [], []
for i in range(len(alice_measures)):
    if alice_measures[i][0] == bob_measures[i][0]:
        alice_match.append(alice_measures[i])
        bob_match.append(bob_measures[i])
        
alice_key = [el[1] for el in alice_match]
bob_key = [el[1] for el in bob_match]
assert alice_key == bob_key

strk = ''.join([str(i) for i in bob_key])
key = bytes(int(strk[i : i + 8], 2) for i in range(0, len(strk), 8))
key_hash = sha256(key).digest()
cipher = AES.new(key_hash, AES.MODE_ECB)
m = b"S3CR3T_M3SS4G3!!"
c = cipher.encrypt(pad(m, 16))
c.hex()

'91a1640e2e7fa7b58742ce5a07de01ca690dd5e318cf97bf82e42585b92ffe01'